In [1]:
import numpy as np
import cvxopt
from sklearn.metrics import confusion_matrix as cm
from sklearn.svm import LinearSVC as svc

In [2]:
#import data
train = np.loadtxt(open("data/data/train.csv", "rb"), delimiter=",")
test = np.loadtxt(open("data/data/test.csv", "rb"), delimiter=",")

#split data
x_train = train[:,1:201]
y_train = train[:,0]
x_test = test[:,1:201]
y_test = test[:,0]

#Convert 0 to -1 for cvxopt
y_train[y_train==0] = -1
y_test[y_test==0] = -1

#soft margin 
soft = 1


In [3]:
#base Parent class
class svm:
    #finds opt line 
    #x = training, y = labbles, C = soft margin
    def fit(self, X, y, C):
        return
    
    #finds projection
    def project(self, X):
        return np.dot(X, self.w) + self.b
    
    #makes prediction
    def predict(self, X):
        return np.sign(self.project(X))


In [4]:
#primal problem
class SVM_prime(svm):
    def fit(self, X, y, C):
        
        n_samples, n_features = X.shape
        n_sum = n_samples + n_features
        
        # P = Diagnal Matix
        temp = np.zeros((n_sum+1, n_sum+1))
        for i in range(n_features):
                temp[i,i] = 1
        P = cvxopt.matrix(temp)
        
        # q = 0(n_samples), C(n_samples)
        q = cvxopt.matrix(np.vstack([np.zeros((n_features+1,1)), C*np.ones((n_samples, 1))]))
        
        # A = -G^Tz
        A = np.zeros((2*n_samples, n_sum+1))
        A[:n_samples,0:n_features] = np.dot(y,X)
        A[:n_samples,n_features] = y.T
        A[:n_samples,n_features+1:]  = np.eye(n_samples)
        A[n_samples:,n_features+1:] = np.eye(n_samples)
        A = cvxopt.matrix(-A)                  
        
        # b = Ax
        b = np.zeros((2*n_samples,1))
        b[:n_samples] = -1
        b = cvxopt.matrix(b)
        
        # solver
        solv = cvxopt.solvers.qp(P, q, A, b)
 
        # Bias
        self.b = solv['x'][n_features]
        
        # Weights
        self.w = np.array(solv['x'][:n_features])
        


In [5]:
#dual problem
class SVM_dual(svm):
    def fit(self, X, y, C):
        
        n_samples, n_features = X.shape
        
        # P = (X^T)X
        temp = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                temp[i,j] = np.dot(X[i], X[j])
        P = cvxopt.matrix(np.outer(y, y) * temp)
        
        # q = -1(1*n_samples)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        
        # G = -1(n_samples*n_samples)
        G = cvxopt.matrix(np.vstack((np.eye(n_samples)*-1,np.eye(n_samples))))
        
        # h = 0(1*n_samples), C(1*n_samples)
        h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
        
        # A = y^T 
        A = cvxopt.matrix(y, (1, n_samples))

        # b = 0 
        b = cvxopt.matrix(0.0)
        
        # solver
        solv = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        # multipliers
        alpha = np.ravel(solv['x'])
        self.a = alpha[alpha > 1e-6]
        self.b = 0
        self.w = np.zeros(n_features)
        
        xx = X[alpha > 1e-6]
        yy = y[alpha > 1e-6]
        aa = np.arange(len(alpha))[alpha > 1e-6]
    

        # Bais
        for i in range(len(self.a)):
            self.b += yy[i]
            self.b -= np.sum(self.a * yy * temp[aa[i], alpha > 1e-6])
        self.b /= len(self.a)
        
        # Weights
        for i in range(len(self.a)):
            self.w += self.a[i] * yy[i] * xx[i]
        

In [6]:
#svm prime
svmP = SVM_prime()
svmP.fit(x_train, y_train, soft)

     pcost       dcost       gap    pres   dres
 0: -8.5000e+03  8.5000e+03  3e+04  3e+00  2e+06
 1:  2.0352e+03 -1.8898e+02  2e+03  3e-02  2e+04
 2:  2.0567e+01 -1.7052e+00  2e+01  3e-04  2e+02
 3:  2.0567e-01 -1.7033e-02  2e-01  3e-06  2e+00
 4:  2.0567e-03 -1.7033e-04  2e-03  3e-08  2e-02
 5:  2.0574e-05 -1.7054e-06  2e-05  3e-10  2e-04
 6:  2.1254e-07 -1.9114e-08  2e-07  3e-12  2e-06
 7:  5.4616e-09 -2.5369e-10  6e-09  3e-14  2e-08
Optimal solution found.


In [7]:
#svm dual 
svmD = SVM_dual()
svmD.fit(x_train, y_train, soft)

     pcost       dcost       gap    pres   dres
 0: -1.6648e+03 -2.3208e+04  2e+05  4e+00  2e-12
 1: -1.0305e+03 -1.4919e+04  3e+04  5e-01  1e-12
 2: -6.7243e+02 -4.4248e+03  6e+03  9e-02  1e-12
 3: -5.3057e+02 -1.7870e+03  2e+03  3e-02  8e-13
 4: -4.8619e+02 -1.0154e+03  8e+02  9e-03  9e-13
 5: -4.9290e+02 -7.2340e+02  3e+02  3e-03  9e-13
 6: -4.9713e+02 -6.4138e+02  2e+02  1e-03  9e-13
 7: -5.1016e+02 -5.9285e+02  9e+01  4e-04  9e-13
 8: -5.0974e+02 -5.8104e+02  8e+01  2e-04  9e-13
 9: -5.1776e+02 -5.6465e+02  5e+01  1e-04  9e-13
10: -5.2185e+02 -5.5515e+02  3e+01  6e-05  9e-13
11: -5.2573e+02 -5.4829e+02  2e+01  3e-05  9e-13
12: -5.2802e+02 -5.4398e+02  2e+01  1e-05  9e-13
13: -5.3068e+02 -5.3987e+02  9e+00  3e-06  1e-12
14: -5.3301e+02 -5.3705e+02  4e+00  1e-06  1e-12
15: -5.3404e+02 -5.3581e+02  2e+00  3e-07  1e-12
16: -5.3458e+02 -5.3519e+02  6e-01  9e-08  1e-12
17: -5.3471e+02 -5.3504e+02  3e-01  3e-08  1e-12
18: -5.3483e+02 -5.3491e+02  8e-02  7e-09  1e-12
19: -5.3486e+02 -5.34

In [8]:
#output prime results
test1 = svmP.predict(x_test)
res1 = cm(y_test, test1)
print(res1)
print(svmP.b)
print(svmP.w)

[[681  78]
 [ 31 710]]
0.00011326512142805203
[[ 3.00181538e-07]
 [ 2.62902183e-07]
 [ 5.97962142e-08]
 [-4.60460050e-07]
 [ 4.72789167e-07]
 [-3.74058901e-07]
 [ 1.63295901e-07]
 [ 3.52872288e-07]
 [ 1.85066525e-07]
 [ 5.65257539e-07]
 [ 3.48881547e-07]
 [-3.55273221e-08]
 [ 1.35166049e-07]
 [ 1.78707297e-07]
 [-2.85840829e-08]
 [-6.85563759e-08]
 [-2.31008706e-08]
 [-5.38133483e-07]
 [-4.41155250e-07]
 [-2.86262614e-07]
 [ 1.32927341e-07]
 [ 1.10348836e-06]
 [ 4.65586367e-07]
 [-3.91676559e-07]
 [-5.61915700e-07]
 [ 8.45193370e-08]
 [ 3.91287219e-08]
 [ 9.06514501e-08]
 [ 3.33243037e-07]
 [ 6.47603058e-07]
 [ 2.94600990e-07]
 [ 6.68335441e-07]
 [-8.25077443e-08]
 [-3.09428375e-07]
 [-1.41817283e-07]
 [ 5.94166072e-07]
 [ 7.98796959e-08]
 [-2.43629828e-07]
 [ 5.23403434e-07]
 [-6.09317908e-08]
 [-1.57163788e-07]
 [ 7.11519793e-07]
 [ 1.07490427e-07]
 [ 4.15296550e-08]
 [-6.54092067e-08]
 [ 4.12084491e-07]
 [ 5.01470585e-07]
 [-1.52524147e-07]
 [ 5.23143873e-07]
 [ 3.15106258e-07]
 [ 2

In [9]:
#output prime results
test2 = svmD.predict(x_test)
res2 = cm(y_test, test2)
print(res2)
print(svmD.b)
print(svmD.w)
print(svmD.a)

[[733  26]
 [ 21 720]]
2.1282549135643567
[-4.23094154e-02 -8.27440372e-02  3.69384960e-02 -1.30016772e-02
  7.80817859e-02 -2.96970660e-02  7.54312967e-02  6.60575778e-03
 -1.06756940e-02  6.90281750e-03  7.19360235e-02 -2.57784488e-02
  4.65686166e-02  6.73030843e-02  3.08061331e-02 -2.18823761e-02
 -5.95650015e-02 -6.49286233e-02 -7.62979003e-02 -4.54316484e-02
 -2.31900838e-02  2.22201646e-02  3.43615759e-02 -1.87006843e-02
 -1.16847302e-02  2.68966834e-02  2.25952777e-02  4.96947086e-02
 -6.15324362e-02  8.61395434e-02 -9.59370593e-02 -1.12216318e-02
  6.45809316e-02  2.65237459e-03  1.22288557e-02 -5.31440409e-02
  1.20391498e-02 -2.92468669e-02  1.14328093e-01 -4.72789793e-02
  2.86409344e-02  5.84477040e-02 -4.36708589e-02  4.14102267e-02
  1.07978861e-02 -2.40829203e-02  6.80745647e-02  1.51174191e-02
  3.20881729e-02  7.13691756e-02 -5.24221736e-02 -4.94304581e-02
  6.43857443e-02 -1.46991384e-02  4.06118309e-02 -1.44634223e-02
 -3.37653432e-02  4.22303894e-02 -1.66736407e-02

In [10]:
#lib prime
libP = svc()
libP.set_params(dual=False)
libP.set_params(C=soft)
#svc.set_params(max_iter=10000)
libP.fit(x_train, y_train)
test3 = libP.predict(x_test)
print(cm(y_test, test3))
print(libP.coef_)


[[736  23]
 [ 15 726]]
[[-1.23003270e-02 -1.85539511e-02  3.01332388e-02 -2.06732517e-02
   3.51744477e-02 -2.56554581e-02  2.92335060e-02 -2.47300320e-02
  -2.58624774e-03 -1.20573290e-02  3.40846534e-02 -4.64239446e-03
   8.57485628e-03  2.55696506e-02  1.03943015e-03 -4.28552659e-03
  -6.50343417e-03 -9.09514044e-03 -3.62258454e-02 -1.09514928e-02
  -1.06560729e-02  1.33332800e-02  2.96973001e-02 -1.56565965e-02
  -5.76449826e-03 -2.83005804e-04  2.28237960e-02  1.46564637e-03
  -3.91145902e-02  3.20977658e-02 -3.56777635e-02 -4.20634976e-03
   1.08510671e-02  4.20161394e-03  1.22100798e-02 -1.31966493e-02
   5.64984895e-03  7.67572157e-03  3.84009412e-02 -1.27388318e-02
   7.62978505e-03  7.57075227e-03 -1.47920786e-02  1.67436597e-02
  -2.92904279e-02 -9.42316948e-03  3.16878865e-02  1.50036765e-03
   1.33432121e-02  3.27791318e-02 -1.59912535e-02 -1.14173471e-02
   3.46321955e-02 -3.11153177e-03  1.49501828e-03 -1.80306581e-02
   3.63195358e-03  4.83294379e-03 -1.21027085e-02  4.

In [12]:
#lib dual
libD = svc()
libD.set_params(dual=True)
libD.set_params(C=soft)
svc.set_params(max_iter=100000)
libD.fit(x_train, y_train)
test4 = libD.predict(x_test)
print(cm(y_test, test4))
print(libD.coef_)


[[734  25]
 [ 15 726]]
[[-1.28552199e-02 -3.38114757e-02  3.51093030e-02 -1.57886252e-02
   3.93657689e-02 -2.31597678e-02  2.84508206e-02 -1.90735876e-02
  -5.22147726e-03 -1.27781590e-02  3.80484364e-02 -3.42704213e-03
   1.03742922e-02  2.17721688e-02  2.70611108e-03  5.14245019e-05
  -2.01024327e-03 -9.30653725e-03 -3.02353641e-02 -3.67772300e-03
  -1.47733964e-02  2.37680420e-02  3.13971037e-02 -1.49204001e-02
  -4.21289904e-03  6.72541134e-03  2.38495034e-02  8.98024044e-03
  -4.21198347e-02  3.77368431e-02 -4.23336112e-02 -1.68613593e-02
   5.04353652e-03  6.84818896e-03  1.53687906e-02 -1.48174719e-02
   3.89116982e-03  6.97025914e-03  2.98557886e-02 -1.34774496e-02
   1.18146920e-02  7.17408566e-03 -1.55605518e-02  1.45562994e-02
  -2.54736461e-02 -9.83638685e-03  2.65577297e-02  1.62779720e-03
   1.24872819e-02  3.12340923e-02 -2.10605607e-02 -2.59207826e-02
   3.72320720e-02 -1.33037343e-03  2.61713018e-03 -1.70268119e-02
   2.82686726e-04  3.63246264e-03 -8.03204301e-03  4.

C:\Users\chopt\Miniconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
